In [15]:
import globals

import os
import urllib
import urllib.request as request
from contextlib import closing

import pickle
import zipfile
import tarfile
from azure.storage.blob import BlockBlobService

from datetime import datetime, timedelta

import numpy as np
import pickle


In [16]:
def downloadAndExtractFile(file):
    local_file = os.path.join(globals.files_dir, file[file.rindex('/')+1:])
    print ("Downloading:" , local_file)
    urllib.request.urlretrieve (file, local_file)
    
    tar = tarfile.open(local_file)
    tar.extractall(globals.files_dir)
    tar.close()

    os.remove(local_file)

def downloadFile(file):
    local_file = os.path.join(globals.files_dir, file[file.rindex('/')+1:])
    print ("Downloading:" , local_file)
    urllib.request.urlretrieve (file, local_file)
    
def uploadFiles(infile, outfile):
    block_blob_service.create_blob_from_path(globals.blob_container_name, outfile, infile)


In [17]:
file_date = str((datetime.now()  - timedelta(days=1)).strftime('%Y-%m-%d'))

In [18]:
globals.resetDir(globals.files_dir)

In [19]:
file = 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_' + file_date + '.tar.gz'
downloadAndExtractFile(file)

file = 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/latest/metadata.csv'
downloadFile(file)

Downloading: /datadrive2/processed/content/cord-19_2020-09-27.tar.gz
Downloading: /datadrive2/processed/content/metadata.csv


In [20]:
local_file = globals.files_dir + '/'+ file_date + '/document_parses.tar.gz'
tar = tarfile.open(local_file)
tar.extractall(os.path.join(globals.files_dir))
tar.close()


In [21]:
block_blob_service = BlockBlobService(account_name=globals.blob_account_name, account_key=globals.blob_account_key)


In [22]:
files = globals.getFilesInDir(os.path.join(globals.files_dir, 'document_parses'))
print ('File Count: ', len(files))

File Count:  194102


In [23]:
# Get list of files existing in blob
block_blob_service = BlockBlobService(account_name=globals.blob_account_name, account_key=globals.blob_account_key)
generator = block_blob_service.list_blobs(globals.blob_container_name, globals.blob_container_path)

# Add Blobs to a list
blobs = []
for blob in generator:
    blobs.append(blob.name)

In [24]:
print ('Total Blob Files:', len(blobs))

Total Blob Files: 194907


In [25]:
tmp_list = []
for b in blobs:
    txt_loc = b.find('json')
    upd_file = os.path.join(globals.files_dir, 'document_parses') + b[txt_loc + len('json'):]
    tmp_list.append(upd_file)

new_files = np.setdiff1d(files, tmp_list)    

In [26]:
print ('Total new files: ', len(new_files))

Total new files:  1941


In [27]:
counter = 0
ovewwrite_if_exists = False
for file in new_files:
    uploadFiles(file, file.replace(os.path.join(globals.files_dir, 'document_parses'), 'json'))
    counter += 1
    if counter % 1000 == 0:
        print ("Uploaded:", counter)
print ("Uploaded:", counter)
    
    

Uploaded: 1000
Uploaded: 1941


In [28]:
#Persist this list for later
with open(os.path.join(globals.files_dir, 'new_files.pkl'), 'wb') as output:
    pickle.dump(new_files, output, pickle.HIGHEST_PROTOCOL)